In [5]:
import numpy as np
import math
from numpy import linalg as LA

***1. Euler2A***

In [6]:
def Euler2A(phi, theta, psi):

    R_z = np.array([[math.cos(psi), -math.sin(psi), 0],
                    [math.sin(psi), math.cos(psi), 0],
                    [0, 0, 1]])

    R_y = np.array([[math.cos(theta), 0, math.sin(theta)],
                    [0, 1, 0],
                    [-math.sin(theta), 0, math.cos(theta)]])

    R_x = np.array([[1, 0, 0],
                    [0, math.cos(phi), -math.sin(phi)],
                    [0, math.sin(phi), math.cos(phi)]])

    A = (R_z.dot(R_y)).dot(R_x)
    return A

In [7]:
A = Euler2A(math.atan2(1, 2), math.asin(2/3) , math.atan2(2, 1))
print(A)

[[ 0.33333333 -0.66666667  0.66666667]
 [ 0.66666667  0.66666667  0.33333333]
 [-0.66666667  0.33333333  0.66666667]]


***2. AxisAngle***

In [8]:
def crossProduct(r, q):
    p = np.zeros((1, 3))
        
    p[0][0] = r[1]*q[2] - r[2]*q[1]
    p[0][1] = r[2]*q[0] - r[0]*q[2]
    p[0][2] = r[0]*q[1] - r[1]*q[0]

    return p

In [9]:
def dotProduct(r, q):
    return r[0]*q[0] + r[1]*q[1] + r[2]*q[2]

In [39]:
def AxisAngle(A):

    # Provera da li je matrica A matrica rotacije
    cond1 = A.dot(A.T).round(5) == np.eye(3)
    cond1 = cond1.all()
    cond2 = np.linalg.det(A) == 1.0    
    if not cond1 or not cond2:
        print("Matrica A nije matrica kretanja")
        return -1, -1

    X = A - np.eye(3)

    r = np.array([X[0][0], X[0][1], X[0][2]])
    q = np.array([X[1][0], X[1][1], X[1][2]])

    p = crossProduct(r, q)
    # ako su r i q linearno zavisni
    if p[0][0] == 0 and p[0][1] == 0 and p[0][2] == 0: 
        q = np.array([X[2][0], X[2][1], X[2][2]])

    norm_p = math.sqrt(p[0][0]**2 + p[0][1]**2 + p[0][2]**2)
    p = p * (1/norm_p) # jedinicni vektor

    # --------------------------------------

    norm_r = math.sqrt(r[0]**2 + r[1]**2 + r[2]**2)
    u = r * (1/norm_r) 

    u_prim = A.dot(u)

    cos_phi = dotProduct(u, u_prim)
    phi = math.acos(cos_phi)

    return p, phi

In [40]:
p, phi1 = AxisAngle(A)
print("Prava: ", p)
print("Ugao: ", phi1)
# izlaz -1, -1 oznacava da A nije matrica rotacije

Prava:  [[1.17756934e-16 7.07106781e-01 7.07106781e-01]]
Ugao:  1.2309594173407745


***3. Rodrigez***

In [52]:
def rodrigez(p, phi):

    # Provera da li je vektor p jedinicni
    cond = (p[0][0]**2+p[0][1]**2+p[0][2]**2) == 1.0
    if not cond:
        print("Vektor p nije jedinicni!")
        return -1

    R = p.T.dot(p)

    Q = np.eye(3) - R

    R = R + math.cos(phi)*Q

    p_x = np.array([[0, -p[0][2], p[0][1]],
                    [p[0][2], 0, -p[0][0]],
                    [-p[0][1], p[0][0], 0]])

    R = R + math.sin(phi)*p_x

    return R

In [53]:
R = rodrigez(p, phi1)
print(R.round(5)) # dobija se matrica iz (1)

[[ 0.33333 -0.66667  0.66667]
 [ 0.66667  0.66667  0.33333]
 [-0.66667  0.33333  0.66667]]


***4. A2Euler***

In [56]:
def A2Euler(A):

    # Provera da li je matrica A ortogonalna
    cond = np.linalg.det(A) == 1.0
    if not cond:
        print("Matrica A nije ortogonalna!")
        return -1, -1, -1

    a = A[2][0]

    if a == 1:
        theta = -math.pi / 2
        phi = 0
        psi = math.atan2(-A[0][1], A[1][1])
    elif a == -1:
        theta = math.pi / 2
        phi = 0
        psi = math.atan2(-A[0][1], A[1][1])
    else:
        theta = math.asin(-a)
        phi = math.atan2(A[2][1], A[2][2])
        psi = math.atan2(A[1][0], A[0][0])
    
    return phi, theta, psi

In [57]:
phi, theta, psi = A2Euler(A)
# dobijaju se pocetni uglovi (osim u slucaju gimbal lock-a)
print("Phi: ", phi) 
print("Theta: ", theta)
print("Psi: ", psi)

Phi:  0.4636476090008061
Theta:  0.7297276562269663
Psi:  1.1071487177940904


***5. AxisAngle2Q***

In [66]:
def AxisAngle2Q(p, phi):

    # Provera da li je vektor p jedinicni
    cond = (p[0][0]**2+p[0][1]**2+p[0][2]**2) == 1.0
    if not cond:
        print("Vektor p nije jedinicni!")
        norm = math.sqrt(p[0][0]**2+p[0][1]**2+p[0][2]**2)
        p = p * (1/norm) # normalizacija

    w = math.cos(phi/2)

    r = math.sin(phi/2) * p

    q = np.array([r, w])

    # sredjivanje kvaterniona (za lakse indeksiranje kasnije)
    q = np.array([q[0][0][0], q[0][0][1], q[0][0][2], q[1]])

    return q    

In [67]:
q = AxisAngle2Q(p, phi1)
print(q)

[6.79869978e-17 4.08248290e-01 4.08248290e-01 8.16496581e-01]


***6. Q2AxisAngle***

In [73]:
def Q2AxisAngle(q):

    # Provera da li je kvaternion jedinicni
    cond = (q[0]**2+q[1]**2+q[2]**2+q[3]**2).round(3) == 1.0
    if not cond:
        print("Kvaternion nije jedincni!")
        norm = math.sqrt(q[0]**2+q[1]**2+q[2]**2+q[3]**2)
        q = q * (1/norm) # normalizacija

    w = q[3]

    if w < 0:
        q = -q

    phi = 2*math.acos(w)

    if abs(w) == 1:
        p = np.array([1, 0, 0])
    else:
        v = np.array([q[0], q[1], q[2]])
        norm_v = math.sqrt(v[0]**2 + v[1]**2 + v[2]**2)
        p = v * (1/norm_v)

    return p, phi

In [74]:
p, phi = Q2AxisAngle(q)
print(p, phi) # rezultat je isti kao i u (2) 

[1.17756934e-16 7.07106781e-01 7.07106781e-01] 1.2309594173407747
